# Modelling 1: Random Forest

In [93]:
import pandas as pd
import numpy as np
from plotnine import *
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [6]:
PATH_DATA = "../data/interim/iterim2.csv"

In [7]:
df = pd.read_csv(PATH_DATA)

In [10]:
df.sample(5)

,Unnamed: 0,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,...,level_q25_x,level_q50_x,level_q75_x,total_records_avg_y,total_records_q25_y,total_records_q50_y,total_records_q75_y,level_q25_y,level_q50_y,level_q75_y
121110,121110,2022-07-30 12:00:00,32.73,KOTA BANDUNG,Abdul Rahman Saleh,2,838.0,82.5,18.99,30,...,2.0,3.0,3.0,16.186992,5.0,12.0,23.0,2.0,2.0,3.0
97677,97677,2022-07-25 16:00:00,32.73,KOTA BANDUNG,Tamansari,1,1250.0,73.0,24.55,33,...,1.0,2.0,3.0,25.903393,6.0,15.0,30.0,1.0,2.0,3.0
154084,154084,2022-08-05 16:00:00,32.73,KOTA BANDUNG,Jalan ABC,2,551.0,93.0,11.83,9,...,2.0,3.0,3.0,8.927438,3.0,7.0,13.0,2.0,2.0,3.0
63837,63837,2022-07-19 12:00:00,32.73,KOTA BANDUNG,Pajajaran,2,990.0,79.0,20.71,37,...,2.0,3.0,3.0,21.199150,7.0,17.0,32.0,2.0,3.0,3.0
77338,77338,2022-07-22 07:00:00,32.73,KOTA BANDUNG,Cijerah Raya,3,530.0,110.5,11.33,10,...,2.0,2.0,3.0,16.432475,5.0,12.0,23.0,2.0,2.0,3.0


In [29]:
sampling_strategy={
  1 : 0.2,
  2 : 0.2,
  3 : 0.2,
  4 : 0.2,
  5 : 0.2,
}

sampling_strategy ='auto'

undersample = RandomUnderSampler(sampling_strategy=sampling_strategy)

In [30]:
X, y = df.drop(columns=['Unnamed: 0']), df['level']

In [31]:
X_over, y_over = undersample.fit_resample(X, y.values)

In [37]:
np.unique(y_over, return_counts=True)

(array([1, 2, 3, 4, 5], dtype=int64),
 array([8501, 8501, 8501, 8501, 8501], dtype=int64))

In [50]:
x_use_cols = ['street',  'median_length', 'median_delay', 'median_speed_kmh',
       'total_records', 'median_level',
       'is_weekend', 'is_holiday', 'hour', 'peak_hour', 'busy_hour',
       'total_records_holiday_sum', 'total_records_holiday_q25',
       'total_records_holiday_q75', 'total_records_holiday_mean',
       'total_records_holiday_q50', 'total_records_busy_hour_mean',
       'total_records_busy_hour_sum', 'total_records_busy_hour_q25',
       'total_records_busy_hour_q50', 'total_records_busy_hour_q75',
       'total_records_avg_x', 'total_records_q25_x', 'total_records_q50_x',
       'total_records_q75_x', 'level_q25_x', 'level_q50_x', 'level_q75_x',
       'total_records_avg_y', 'total_records_q25_y', 'total_records_q50_y',
       'total_records_q75_y', 'level_q25_y', 'level_q50_y', 'level_q75_y']
y_use_cols = ['level']

In [51]:
X_bal = df[x_use_cols]
y_bal = df[y_use_cols]

In [52]:
X_bal.sample(3)

,street,median_length,median_delay,median_speed_kmh,total_records,median_level,is_weekend,is_holiday,hour,peak_hour,...,level_q25_x,level_q50_x,level_q75_x,total_records_avg_y,total_records_q25_y,total_records_q50_y,total_records_q75_y,level_q25_y,level_q50_y,level_q75_y
113394,Diponegoro,438.0,97.0,11.96,11,3.0,0,0,20,0,...,2.0,3.0,4.0,16.750829,4.25,11.0,21.0,2.0,3.0,4.0
143288,Cibolerang,1031.0,103.0,13.48,3,1.0,0,0,15,1,...,1.0,2.0,2.0,10.646825,4.00,9.0,14.0,1.0,2.0,3.0
103888,Gede Bage,488.0,76.0,10.64,1,2.0,0,0,21,0,...,1.0,2.0,3.0,10.870421,4.00,9.0,15.0,1.0,2.0,3.0


In [54]:
y_bal.sample(3)

,level
265287,3
291191,5
54130,2


In [71]:
# Label encoder
street_label_dic = {'undefined': 0}
street_label = [0]
street_encoded = []

In [87]:
for index, rows in X_bal.iterrows():
  if not rows['street'] in street_label_dic:
    street_label.append((max(street_label) + 1))   
    street_label_dic[rows['street']] = max(street_label)
  street_encoded = street_label_dic[rows['street']]

In [88]:
X_bal['street'] = street_encoded

<ipython-input-88-d03f8c17f22d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [89]:
X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal, test_size=0.33, random_state=42)

In [90]:
clf = RandomForestClassifier(max_depth=10, random_state=42, n_estimators=35)

In [91]:
clf.fit(X_train,y_train)

<ipython-input-91-86937c1966f0>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestClassifier(max_depth=10, n_estimators=35, random_state=42)

In [92]:
y_pred = clf.predict(X_test)

In [94]:
metrics.accuracy_score(y_pred,y_test)

0.9986052438816364

In [98]:
metrics.confusion_matrix(y_test, y_pred)

array([[13060,    98,     4,     0,     0],
       [    0, 33946,     0,     0,     0],
       [    0,     0, 33348,     0,     0],
       [    0,     0,    37, 16336,     0],
       [    0,     0,     0,     0,  2830]], dtype=int64)